# Intertidal elevation modelling case study

### Background
Intertidal environments support important ecological habitats (e.g. sandy beaches and shores, tidal flats and rocky shores and reefs), and provide many valuable benefits such as storm surge protection, carbon storage and natural resources for recreational and commercial use. However, intertidal zones are faced with increasing threats from coastal erosion, land reclamation (e.g. port construction), and sea level rise. Accurate elevation data describing the height and shape of the coastline is needed to help predict when and where these threats will have the greatest impact. However, this data is expensive and challenging to map across the entire intertidal zone of a continent the size of Australia.

### Digital Earth Australia use case
The rise and fall of the tide can be used to reveal the three-dimensional shape of the coastline by mapping the boundary betweeen water and land across a range of known tides (e.g. from low tide to high tide). Assuming that the land-water boundary is a line of constant height relative to mean sea level (MSL), elevations can be modelled for the area of coastline located between the lowest and highest observed tide. 

Imagery from satellites such as the NASA/USGS Landsat program is available for free for the entire planet, making satellite imagery a powerful and cost-effective tool for modelling the 3D shape and structure of the intertidal zone at regional or national scale. Recently, Geoscience Australia combined 30 years of Landsat data from the Digital Earth Australia archive with tidal modelling to produce the first 3D model of Australia's entire coastline: the **National Intertidal Digital Elevation Model** or NIDEM (for more information, see [Bishop-Taylor et al. 2019](https://doi.org/10.1016/j.ecss.2019.03.006)). 

In this example, we demonstrate a simplified version of the NIDEM method that combines data from the Landsat 5, 7 and 8 satellites with tidal modelling, image compositing and spatial interpolation techniques. We first map the boundary between land and water from low to high tide, and use this information to generate smooth, continuous 3D elevation maps of the intertidal zone. The resulting data may assist in mapping the habitats of threatened coastal species, identifying areas of coastal erosion, planning for extreme events such as storm surges and flooding, and improving models of how sea level rise will affect the Australian coastline. This worked example takes users through the code required to:

1.  Load in a cloud-free Landsat time series
2.  Compute a water index (NDWI)
3.  Tag and sort satellite images by tide height
4.  Create "summary" or composite images that show the distribution of land and water at discrete intervals of the tidal range (e.g. at low tide, high tide)
5.  Extract and visualise the topography of the intertidal zone as depth contours
6.  Interpolate depth contours into a smooth, continuous Digital Elevation Model (DEM) of the intertidal zone

### Technical details

* Products used: `ga_ls5t_ard_3`, `ga_ls7e_ard_3`, `ga_ls8c_ard_3`
* Bands used: `red`, `green`, `blue` and `nir`. You can read more about the bands available for Landsat [here](https://landsat.gsfc.nasa.gov/landsat-8/landsat-8-bands/)
* Analyses used: NDWI water index, tide tagging, image compositing, waterline extraction, spatial interpolation

**To run this analysis, run all the cells in the notebook, starting with the "Load modules" cell. When you finished the analysis, you can return to the "Set up analysis" cell, modify some values (e.g. choose a different location, time, tide height range or time period to analyse) and re-run the analysis. There are additional instructions on modifying the notebook at the end.**

## Load modules

In [ ]:
import datacube
import sys
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import waterline_funcs
from utils.display import display_map

## Set up analysis

The following lines allow us to set up our analysis by selecting the location (`lat_range` and `lon_range`) and time (`time_range`) we want to investigate. We also need to provide a file containing tide heights (`tide_file`) for a coastal location located near the area we wish to analyse.

* **lat_range:** The latitude range to analyse (e.g. `(-12.20, -12.27)`). For fast load times, keep this to a range of ~0.1 degrees or less.

* **lon_range:** The longitude range to analyse (e.g. `(131.80, 131.91)`). For fast load times, keep this to a range of ~0.1 degrees or less.

* **time_range:** The date range to analyse (e.g. `('2013-01-01', '2018-12-31') `)

* **tide_file:** A text file containing a list of times and tide heights for a location near to the study area. These tides are used to filter the satellite dataset to focus on images taken at certain tide conditions. Tide height files are stored in the `tide_heights` folder and are named using the following convention: `locationname_latitude_longitude_tides.csv`.

### Edit these lines to change any of the analysis parameters

If you're running this notebook for the first time, we recommend you keep the default settings below. This will allow you to understand how the analysis works.

The example we've selected looks at the intertidal flats in Southern Moreton Bay south of Brisbane, Australia. **To change the location**, you'll need to make sure Landsat 5, 7 and 8 data is available for the new location, which you can check at the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all Landsat products). You will need to find tide heights for your region and store them in the same format as the `data/tide_heights/moretonbay_-27.552_153.355_tides.csv`. You can choose from some existing tide height files in the `data/tide_heights` folder.

Run the following cell to set values for the various parameters discussed above.

In [ ]:
lat_range = (-27.60, -27.665)    
lon_range = (153.33, 153.425) 
time_range = ('1987', '2019') 
tide_file = 'data/tide_heights/moretonbay_-27.552_153.355_tides.csv'

### View the selected location

The next cell will display the selected area on an interactive map. Feel free to zoom in and out to get a better understanding of the area you'll be analysing. Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(latitude=lat_range, longitude=lon_range)

## Load cloud-masked Landsat data
The first step in this analysis is to load in Landsat data for the `lat_range`, `lon_range` and `time_range` we provided above. The code below first connects to the datacube database, and then uses the `load_cloudmaskedlandsat` function to load in data from the Landsat 5, 7 and 8 satellites for the area and time included in `lat_range`, `lon_range` and `time_range`. The function will also automatically mask out clouds from the dataset, allowing us to focus on pixels that contain useful data:

In [ ]:
# Connect to the datacube
dc = datacube.Datacube(app='Intertidal elevation')

# Create the 'query' dictionary object, which contains the longitudes, latitudes and time provided above
query = {
    'y': lat_range,
    'x': lon_range,
    'time': time_range,
    'output_crs': 'EPSG:28356',
    'resolution': (-30, 30)
}

# Load satellite data for all Landsat satellites (5, 7 and 8) from 1987 onward
landsat_ds = waterline_funcs.load_cloudmaskedlandsat(
    dc, 
    query, 
    bands=['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_nir']
)

print(landsat_ds)

### Plot example timestep in true colour
To verify that we extracted data for the area we're interested in analysing, we can plot up the first image in the dataset in true colour. This makes use of the `red`, `green` and `blue` Landsat bands that we loaded in the previous cell.

Change `index=0` to another value (e.g. `index=5`) to plot up different timesteps. You should be able to clearly see where clouds have been masked out.

In [ ]:
waterline_funcs.rgb(landsat_ds, index=0)

## Compute Normalised Difference Water Index
To extract intertidal depth contours, we need to be able to seperate water from land in our study area. To do this, we can use our Landsat data to calculate a water index called the `Normalised Difference Water Index`, or NDWI. This index uses the ratio of green and near-infrared radiation to identify the presence of water. The formula is:

$$
\begin{aligned}
\text{NDWI} &= \frac{(Green - NIR)}{(Green + NIR)}
\end{aligned}
$$

where `Green` is the green band and `NIR` is the near-infrared band. 

When it comes to interpreting the index, **High values (greater than 0, blue colours) typically represent water pixels**, while **low values (less than 0, red colours) represent land**. You can use the cell below to calculate and plot one of the images after calculating the index.

In [ ]:
landsat_ds['water_index'] = ((landsat_ds.green - landsat_ds.nir) / (landsat_ds.green + landsat_ds.nir))
landsat_ds.water_index.isel(time=0).plot(cmap='RdBu', size=6, vmin=-0.8, vmax=0.8)
plt.show()

How does the plot of the index compare to the optical image from earlier? Was there water or land anywhere you weren't expecting?

## Import tide height data
The location of the shoreline can vary greatly from low to high tide. In the code below, we aim to calculate the height of the tide at the exact moment each Landsat image was acquired. This will allow us to built a sorted time series of images taken at low tide to high tide, which we will use to generate the intertidal elevation model.

You can load the existing tide height data in the following cell using the `pandas` module, which we imported earlier. The code will read the `tide_file` variable provided during the set up. The data below consists of hourly tide heights for the entire period between 1987 and 2019.

In [ ]:
tide_data = pd.read_csv(tide_file, parse_dates=['time'], index_col='time')
tide_data.head()

Now that we have the tide height data, we need to estimate the tide height for each of our Landsat images. We can do this by interpolating between the data points we do have (hourly measurements) to get the tide height for the exact moment each satellite image was taken.

In [ ]:
# First, we convert the data to an xarray dataset so we can analyse it in the same way as our Landsat data
tide_data_xr = tide_data.to_xarray()

# We want to convert our hourly tide heights to estimates of exactly how high the tide was at the time that
# each satellite image was taken. To do this, we can use `.interp` to 'interpolate' a tide height for each
# Landsat timestamp:
landsat_tideheights = tide_data_xr.interp(time=landsat_ds.time)

# We then want to put these values back into the Landsat dataset so that each image has an estimated tide height:
landsat_ds['tide_height'] = landsat_tideheights.tide_height

# Plot the resulting tide heights for each Landsat image:
landsat_ds.tide_height.plot()
plt.show()

## Create water index summary images from low to high tide

Using these tide heights, we can sort our Landsat dataset by tide height to reveal which parts of the landscape are inundated or exposed from low to high tide. 

Individual remote sensing images can be affected by noise, including clouds, sunglint and poor water quality conditions (e.g. sediment). To produce cleaner images that can be compared more easily between tidal stages, we can create 'summary' images or composites that combine multiple images into one image to reveal the 'typical' or median appearance of the landscape at different tidal stages. In this case, we use the median as the summary statistic because it prevents strong outliers (like stray clouds) from skewing the data, which would not be the case if we were to use the mean.

In the code below, we take the time series of images, sort by tide and categorise each image into 9 discrete tidal intervals, ranging from the lowest (tidal interval 1) to the highest tides observed by Landsat (tidal interval 9). For more information on this method, refer to [Sagar et al. 2018](https://www.sciencedirect.com/science/article/pii/S0034425717301591).

In [ ]:
# Sort every image by tide height
landsat_ds = landsat_ds.sortby('tide_height')

# Bin tide heights into 9 tidal intervals from low (1) to high tide (9)
binInterval = landsat_ds.tide_height.quantile(np.linspace(0, 1.0, 10)).values
tide_intervals = pd.cut(
    landsat_ds.tide_height,
    bins = binInterval,
    labels=range(1, 10),
    include_lowest=True
)
landsat_ds['tide_interval'] = xr.DataArray(tide_intervals, [('time', landsat_ds.time)])

print(landsat_ds)

Now that we have a dataset where each image is classified into a discrete range of the tide, we can combine our images into a set of nine individual images that show where land and water is located from low to high tide. **This step can take several minutes to process.**

In [ ]:
# For each interval, compute the median water index and tide height value
landsat_intervals = (landsat_ds[['tide_interval', 'water_index', 'tide_height']]
                     .compute()
                     .groupby('tide_interval')
                     .median(dim='time'))

# Plot the resulting set of tidal intervals
landsat_intervals.water_index.plot(col='tide_interval', col_wrap=5, cmap='RdBu')
plt.show()

The plot above should make it clear how the shape and structure of the coastline changes significantly from low to high tide as low-lying tidal flats are quickly inundated by increasing water levels.

## Extract depth contours from imagery
We now want to extract an accurate boundary between land and water for each of the tidal intervals above. The code below identifies the depth contours based on the boundary between land and water by tracing a line along pixels with a water index value of `0` (halfway between land and water water index values). It returns a shapefile with one depth contour for each tidal interval that is labelled with tide heights in metres relative to Mean Sea Level:


In [ ]:
# Set up attributes to assign to each waterline
attribute_data = {'tide_m': landsat_intervals.tide_height.values}
attribute_dtypes = {'tide_m': 'float'}

# Extract waterline contours for the '0' water index threshold:
contour_gdf = waterline_funcs.contour_extract(
    z_values=[0],
    ds_array=landsat_intervals.water_index,
    ds_crs=landsat_ds.crs,
    ds_affine=landsat_ds.geobox.transform,
    output_shp=f'output_depthcontours.shp',
    attribute_data=attribute_data,
    attribute_dtypes=attribute_dtypes,
    min_vertices=5,
    dim='tide_interval'
)

# Plot output shapefile over the top of the first tidal interval water index
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
landsat_intervals.water_index.sel(tide_interval=1).plot(ax=ax, cmap='Greys')
contour_gdf.plot(cmap='YlOrRd', ax=ax)
plt.show()

The above plot is a basic visualisation of the depth contours returned by the `contour_extract` function. Given we now have the shapefile, we can use a more complex function to make an interactive plot for viewing the topography of the intertidal zone.

### Plot interactive map of depth contours coloured by time
The next cell provides an interactive map with an overlay of the depth contours identified in the previous cell. Run it to view the map.

Zoom in to the map below to explore the resulting set of depth contours. Deeper contours are coloured in yellow; more shallow contours are coloured in red. Using this data, we can easily identify areas of the coastline which are only exposed in the lowest of tides, or other areas that are only covered by water during high tides.

In [ ]:
waterline_funcs.map_shapefile(gdf=contour_gdf)

## Interpolate contours into a Digital Elevation Model (DEM)

While the contours above provide valuable information about the topography of the intertidal zone, we can extract additional information about the 3D structure of the coastline by converting them into an elevation raster (i.e. a Digital Elevation Model or DEM).

In the cell below, we convert the shapefile above into an array of points with X, Y and Z coordinates, where the Z coordinate is the point's elevation relative to Mean Sea Level. We then use these XYZ points to interpolate smooth, continuous elevations across the intertidal zone using linear interpolation.

In [ ]:
# First convert our contours shapefile into an array of XYZ points
xyz_array = waterline_funcs.contours_to_arrays(contour_gdf, 'tide_m')

# Interpolate these XYZ points over the spatial extent of the Landsat dataset
intertidal_dem = waterline_funcs.interpolate_2d(
    ds=landsat_intervals,
    x_coords=xyz_array[:,0],
    y_coords=xyz_array[:,1],
    z_coords=xyz_array[:,2]
)

# Plot the output
intertidal_dem.plot(cmap='viridis', size=8, robust=True)
plt.show()

You can see in the output above that our interpolation results are very messy. This is because the interpolation extends across areas of our study area that are not affected by tides (e.g. areas of water located beyond the lowest observed tide, and on land). To clean up the data, we can restrict the DEM to only the area between the lowest and highest observed tides:

In [ ]:
# Identify areas that are always wet (e.g. below low tide), or always dry
above_lowest = landsat_intervals.isel(tide_interval=0).water_index < 0
below_highest = landsat_intervals.isel(tide_interval=-1).water_index > 0

# Keep only pixels between high and low tide
intertidal_dem_clean = intertidal_dem.where(above_lowest & below_highest)

# Plot the cleaned dataset
intertidal_dem_clean.plot(cmap='viridis', size=8, robust=True)
plt.show()

## Next steps
When you are done, return to the "Set up analysis" cell, modify some values (e.g. `time_range`, `lat_range`, `lon_range`) and rerun the analysis. 

If you're going to change the location, you'll need to make sure Landsat 5, 7 and 8 data is available for the new location, which you can check at the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all Landsat products). You will  need to find tide heights for your region and store them in the same format as the `data/tide_heights/moretonbay_-27.552_153.355_tides.csv`. You will also need to update the `tide_file` variable in the set up. You can choose from some existing tide height files in the `data/tide_heights` folder.

## Further exploration
For more information about the science behind this notebook, please refer to the scientific article outlining the application of this approach to the entire Australian coastline: [Bishop-Taylor et al. 2019 *Between the tides: Modelling the elevation of Australia's exposed intertidal zone at continental scale*](https://doi.org/10.1016/j.ecss.2019.03.006).

While this notebook mostly utilised the functions available from the Open Data Cube library, some of the steps were conducted with functions from the `waterline_funcs` module, which you can find in the `utils` folder. If you're curious about how these functions work, feel free to explore the `utils/waterline_funcs.py` file. This option is for users who feel comfortable programming in Python. You'll be able to navigate to this file from the Jupyter file explorer page.